# Oficina de Ferramentas IV: OCR e leitura distante com NLP para a História

## 1. OCR: o que é e como funciona?

Chamamos de reconhecimento ótico de caracteres o processo computacional de aplicar um programa de computador para encontrar padrões de escrita em imagens e gerar um documento de texto como resultado. Esse processo é muito importante para as pesquisas históricas no geral, visto que parte significativa das fontes históricas utilizadas em nossas pesquisas ainda são constituídas por documentos escritos (impressões ou manuscritos) em papel.

Desde os anos 1990, vemos o crescimento acelerado de projetos de digitalização em massa de documentos históricos. É muito recorrente que o objeto digital oriundo desse processo de digitalização seja uma imagem estática em formatos variados.

Entretanto, para que possamos aplicar técnicas e ferramentas digitais, sejam elas bem simples como a busca de um termo, ou mais complexas como a aplicação de técnicas de processamento de linguagem natural ou modelagem de tópicos, precisamos transformar essas imagens em documentos de texto, onde os dados contidos nela possam ser estruturados de acordo com as metodologias e interesses das pesquisas.

Para que isso seja possível, precisamos realizar o reconhecimento ótico de caracteres, ou OCR (optical character recognition), nesses conjuntos de imagens.

Portanto, essa é uma etapa fundamental para pesquisas com jornais históricos digitalizados. Entretanto, esse não é um processo simples e demanda muito conhecimento computacional, máquinas potentes, e investimento de tempo e capital.

Realizar esses processos de forma individualizada e sem acompanhamento de profissionais especializados muitas vezes se torna um desafio quase intransponível. O que é potencializado para o caso dos jornais históricos, pois eles agrupam características que impõem grandes entraves para o sucesso do OCR. Podemos listar alguns deles:

- baixa qualidade da digitalização;
- fontes muito variadas e desconhecidas para o computador;
- layouts complexos, geralmente com grandes variações em uma mesma edição e com muitas colunas pouco claras;
- muitos “ruídos” na imagem, como rasgos, rasuras, manchas, anotações, cortes, etc;


### 1.1. Como funciona o OCR?

O OCR é um processo que envolve diversas etapas, que podem ser resumidas em:

1. **Pré-processamento**: é a etapa onde a imagem é tratada para que o OCR possa ser aplicado. Isso envolve a remoção de ruídos, ajuste de contraste, binarização, entre outros.
2. **Segmentação**: é a etapa onde a imagem é dividida em regiões que contém texto. Isso é feito para que o OCR possa ser aplicado em cada região separadamente.
3. **Reconhecimento de caracteres**: é a etapa onde o OCR de fato acontece. Aqui, o programa tenta identificar os caracteres presentes na imagem.
4. **Pós-processamento**: é a etapa onde o texto reconhecido é tratado para corrigir erros e melhorar a qualidade do resultado final.



---

## 2. OCR com Tesseract

O Tesseract é uma das ferramentas de OCR mais populares e amplamente utilizadas. Ele é um software livre e de código aberto e é capaz de reconhecer mais de 100 idiomas. O Tesseract atualmente é mantido pelo Google e é amplamente utilizado em diversos projetos de OCR. Veja um breve histórico [aqui](https://github.com/tesseract-ocr/tesseract?tab=readme-ov-file#brief-history).

Vamos testar o tesseract com uma imagem simples de um texto através da linha de comando.

#### Instalação do Tesseract

**Linux**: 

```bash
sudo apt install tesseract-ocr
sudo apt install libtesseract-dev
```

**Windows**: 

Acesse a página e baixe o instalador: [https://github.com/UB-Mannheim/tesseract/wiki#tesseract-installer-for-windows](https://github.com/UB-Mannheim/tesseract/wiki#tesseract-installer-for-windows)


### OCR com Tesseract

```bash
tesseract INPUT_FILENAME.tiff OUTPUT_FILENAME -l por
```


Exportar para hOCR:

```bash 
tesseract INPUT_FILENAME.tiff OUTPUT_FILENAME -l por hocr
```


### Pré-processamento com ImageMagick

```bash
convert -density 300 INPUT_FILENAME.pdf -depth 8 -strip -background white -alpha off OUTPUT_FILENAME.tiff
```


---


## 3. OCR com Python

Vamos criar um script em python para transformar cada página do pdf em tiff e depois aplicar o OCR com o Tesseract.

Vamos testar com o livro de Lima Barreto, Triste fim de Policarmo Quaresma.

http://objdigital.bn.br/objdigital2/acervo_digital/div_obrasraras/or22535/or22535.pdf

http://acervo.bndigital.bn.br/sophia/index.asp?codigo_sophia=56881

Primeiro instale as bibliotecas necessárias:

```bash
pip install pdf2image pytesseract pillow numpy
```

Um script Python é um arquivo de texto com extensão `.py` que contém um conjunto de instruções que serão executadas pelo interpretador Python.

As instruções serão executadas de cima para baixo, uma a uma, até o final do arquivo.

Primeiro precisamos importar as bibliotecas necessárias para nosso script.

In [2]:
import os # biblioteca para manipulação de arquivos
import pdf2image # biblioteca para converter pdf em imagem
import pytesseract # biblioteca para reconhecimento de texto em imagem

Agora vamos criar uma função para converter cada página do pdf em uma imagem tiff.

Para isso, vamos utilizar a biblioteca pdf2image.

In [3]:
def pdf2img(pdf_path, img_path):
    '''
    Converte um arquivo PDF em imagens TIFF.
    :param pdf_path: caminho do arquivo PDF
    :param img_path: caminho do diretório de destino das imagens

    Exemplo:
    pdf2img("arquivo.pdf", "imagens")
    '''
    # se o diretório de destino não existir, cria
    if not os.path.exists(img_path):
        os.makedirs(img_path)

    # converte o PDF em imagens, com resolução de 300 DPI e escala de cinza
    # a variável images é uma lista com as imagens geradas
    images = pdf2image.convert_from_path(
        pdf_path,
        dpi=300,
        fmt="tiff",
        grayscale=True)       

    # salva as imagens no diretório de destino com o nome 0.tiff, 1.tiff, ...
    for i, image in enumerate(images):
        image.save(f"{img_path}/{i}.tiff", "TIFF")

    print(f"Arquivo {pdf_path} convertido com sucesso.")

In [4]:
# Função para salvar o texto extraído em um arquivo
def save_text(text, txt_path):
    '''
    Salva um texto em um arquivo TXT.
    :param text: texto a ser salvo
    :param txt_path: caminho do arquivo TXT

    Exemplo:
    save_text("Olá, mundo!", "arquivo.txt")
    '''
    with open(txt_path, "w") as file:
        file.write(text)
    print(f"Texto salvo em {txt_path}")

In [6]:
# função para aplicar o OCR nas imagens com idioma por usando pytesseract
def ocr(img_path, lang):
    '''
    Aplica OCR em imagens.
    :param img_path: caminho do diretório com as imagens
    :param lang: idioma

    Exemplo:
    ocr("imagens", "por")
    '''
    # para cada imagem no diretório img_path aplica o OCR
    for img in os.listdir(img_path):
        text = pytesseract.image_to_string(
            img_path + "/" + img, # imagem a ser lida
            lang=lang, # idioma
            config='--psm 6') # psm 6 é para tratar texto em bloco
        print(f"Texto extraído da imagem {img}")
        # salva o texto extraído em um arquivo
        save_text(text, f"{img_path}/{img}.txt")

In [7]:
pdf_path = "assets/lima_barreto_dominio_publico.pdf"
img_path = "assets/images_limabarreto"
lang = "por"


In [ ]:
# chama a função pdf2img para converter o PDF em imagens
pdf2img(pdf_path, img_path)


In [ ]:
# chama a função ocr para aplicar o OCR nas imagens
ocr(img_path, lang)

In [5]:
# função para unir todos os textos extraídos em um único arquivo, respeitando a ordem das páginas
# reuna apenas os arquivos .txt classificados em ordem crescente
def merge_txt_files(txt_path, output_path):
    '''
    Une arquivos TXT em um único arquivo.
    :param txt_path: caminho do diretório com os arquivos TXT
    :param output_path: caminho do arquivo de saída

    Exemplo:
    merge_txt_files("imagens", "output.txt")
    '''
    # lista os arquivos .txt no diretório txt_path
    files = [f"{txt_path}/{file}" for file in os.listdir(txt_path) if file.endswith(".txt")]
    # ordena os arquivos pelo nome (0.txt, 1.txt, ...)
    # para isso utilizamos uma função lambda que extrai o número do nome do arquivo e converte para inteiro
    files.sort(key=lambda x: int(x.split("/")[-1].split(".")[0]))
    
    # para cada arquivo, lê o conteúdo e salva no arquivo de saída
    with open(output_path, "w") as output:
        for file in files:
            print(f"Unindo {file}")
            with open(file, "r") as f:
                output.write(f.read())
    print(f"Arquivos unidos com sucesso em {output_path}")

In [ ]:
merge_txt_files(img_path, "assets/quaresma.txt")